In [4]:
from pathlib import Path
import os
# Constants.
DATASET_DIR = Path(os.getcwd()) / "datasets"

In [5]:
# Wanb init.
def init_wandb():
    try:
        import wandb
        # Start a new wandb run to track this script.
        return wandb.init(
            # Set the wandb entity where your project will be logged (generally your team name).
            entity="prinzz-personal",
            # Set the wandb project where this run will be logged.
            project="gadgets-predictor",
            # Track hyperparameters and run metadata.
            config={
                "learning_rate": 0.02,
                "architecture": "CNN",
                "dataset": "images",
                "epochs": 10,
            },
        )
    except ImportError:
        print("wandb is not installed. Skipping wandb initialization.")
    except Exception as e:
        print(f"An error occurred during wandb initialization: {e}")


In [6]:
from duckduckgo_search import DDGS
import requests
import dask
from fastai.vision.all import get_image_files
from dask.distributed import Client

def download_images(query, output_dir, max_results=50):
    os.makedirs(output_dir, exist_ok=True)

    # Check if output_dir already has enough images
    existing_files = [
        f for f in os.listdir(output_dir)
        if os.path.isfile(os.path.join(output_dir, f))
    ]

    if len(existing_files) >= max_results:
        print(f"Skipping '{query}': {len(existing_files)} images already present.")
        return

    ddg = DDGS()
    results = ddg.images(query, max_results=max_results)

    downloaded = len(existing_files)
    
    for idx, result in enumerate(results):
        if downloaded >= max_results:
            break
        image_url = result["image"]
        try:
            response = requests.get(image_url, timeout=10)
            response.raise_for_status()
            ext = image_url.split(".")[-1].split("?")[0][:4]
            filename = os.path.join(output_dir, f"{query.replace(' ', '_')}_{downloaded}.{ext}")
            with open(filename, "wb") as f:
                f.write(response.content)
            print(f"Downloaded: {filename}")
            downloaded += 1
        except Exception as e:
            print(f"Failed to download {image_url}: {e}")

client = Client(threads_per_worker=os.cpu_count() // 2, n_workers=os.cpu_count())
gadgets = ["smartphone", "tablet", "smartwatch", "headphones", "camera"]
parallel_results = []
for gadget in gadgets:
    parallel_result= dask.delayed(download_images)(gadget, output_dir=DATASET_DIR / gadget, max_results=200)
    parallel_results.append(parallel_result)
# parallel_results = dask.compute(*parallel_results)
print("All downloads completed.")




All downloads completed.


In [7]:
from fastai.vision.all import ImageDataLoaders, Resize

# Dataloaders.
dls = ImageDataLoaders.from_folder(DATASET_DIR,
                                   train_pct=0.8,
                                   valid_pct=0.2,
                                   item_tfms=Resize(224))


In [9]:
from fastai.vision.all import vision_learner,resnet18,error_rate,accuracy
learn = vision_learner(dls, resnet18, metrics=[accuracy,error_rate],pretrained=True)
learn.fine_tune(1)

epoch,train_loss,valid_loss,accuracy,error_rate,time
0,1.752798,0.337808,0.883436,0.116564,00:20


/Users/Priyanshu/Documents/my-projects/ml-learning/virtual-env/lib/python3.13/site-packages/PIL/Image.py:1043: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


epoch,train_loss,valid_loss,accuracy,error_rate,time
0,0.366446,0.256415,0.926380,0.073620,00:21


In [10]:
learn.model.eval()


Sequential(
  (0): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  